In [ ]:
import pandas as pd
import numpy as np

In [ ]:
diabetes_df = pd.read_csv( "HBA1C_Report_YESH.csv", sep = ",",encoding = "ISO-8859-1")

In [ ]:
diabetes_df.head( 3 )


In [ ]:
diabetes_df.shape

In [ ]:
diabetes_df.dtypes

In [ ]:
diabetes_df['createddate'] = pd.to_datetime(diabetes_df['createddate'])
diabetes_df['DOB'] = pd.to_datetime(diabetes_df['DOB'])
diabetes_df.info()

In [ ]:
diabetes_df.isnull().sum(axis=0)

In [ ]:
diabetesPatient = diabetes_df[['patientfirstname','DOB','patientid']].copy()
diabetesPatient = diabetesPatient.groupby(['patientfirstname','DOB']).count()


In [ ]:
import warnings
warnings.filterwarnings('ignore')

diabetesPatient.sort_index(by=['patientid'],ascending=False).to_csv('DiabetesGrouping.csv', sep=',', encoding='utf-8')


In [ ]:
diabetes_df.sort_values(by=['patientfirstname','DOB','createddate'],ascending=False, inplace=True)


In [ ]:
diabetes_df_copy = pd.DataFrame(columns=diabetes_df.columns)
diabetes_df_copy["prediabetesdate"] = ""
diabetes_df_copy["prediabetesresult"] = ""

In [ ]:
groupKey = diabetes_df.iloc[0]['patientfirstname'] + row['patientid'] + str(diabetes_df.iloc[0]['DOB'])

In [ ]:
diabetes_df_copy.shape

In [ ]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [ ]:
from datetime import datetime
from dateutil import relativedelta

foundDiabetic = False
foundPreDiabetic = False
found1 = False
found2 = False
foundRecord = False
diabeticRow = ""
found1Row = ""
found2Row = ""
f1Row = ""
f2Row = ""


In [ ]:
for index, row in diabetes_df.iterrows():
    currentKey = row['patientfirstname'] + row['patientid'] + str(row['DOB'])
    if currentKey != groupKey:
        groupKey = currentKey
        
        if foundRecord == False and found1 == True and found2 == True:
            found1Row = pd.Series.to_frame(f1Row).T
            found2Row = pd.Series.to_frame(f2Row).T
            
            found1Row["prediabetesdate"] = f2Row['createddate']
            found1Row["prediabetesresult"] = f2Row['result']
            difference = relativedelta.relativedelta(f1Row['createddate'], f1Row['DOB'])
            age =  float(difference.years)
            found1Row["age"] = age
            found1Row["agegroup"] = ""
            
            if age >= 18 and age <= 29:
               found1Row["agegroup"] = "18-29"
            if age >= 30 and age <= 39:
               found1Row["agegroup"] = "30-39"
            if age >= 40 and age <= 50:
               found1Row["agegroup"] = "40-50"
            if age >= 51:
               found1Row["agegroup"] = "51+"
            found1Row["isdiabetic"] = 0
            diabetes_df_copy = diabetes_df_copy.append(found1Row)
        
        foundDiabetic = False
        foundPreDiabetic = False
        found1 = False
        found2 = False
        foundRecord = False
        
    if isfloat(row['result']):
        
        if found1 == True and found2 == False and float(row['result']) > 5.6 and float(row['result']) < 6.5:
            f2Row = row
            found2 = True
        
        if found1 == False:
            f1Row = row
            found1 = True
            

        
        if float(row['result']) >= 6.5 and foundDiabetic == False:
            foundDiabetic = True
            diabeticRow = pd.Series.to_frame(row).T
            diabeticRow["isdiabetic"] = 1

        if float(row['result']) > 5.6 and float(row['result']) < 6.5 and foundDiabetic == True and foundPreDiabetic == False:
            foundPreDiabetic = True
            diabeticRow["prediabetesdate"] = row['createddate']
            diabeticRow["prediabetesresult"] = row['result']
            difference = relativedelta.relativedelta(row['createddate'], row['DOB'])
            age =  float(difference.years)
            diabeticRow["age"] = age
            diabeticRow["agegroup"] = ""
            
            if age >= 18 and age <= 29:
               diabeticRow["agegroup"] = "18-29"
            if age >= 30 and age <= 39:
               diabeticRow["agegroup"] = "30-39"
            if age >= 40 and age <= 50:
               diabeticRow["agegroup"] = "40-50"
            if age >= 51:
               diabeticRow["agegroup"] = "51+"
            
            diabetes_df_copy = diabetes_df_copy.append(diabeticRow) 
            foundRecord = True

In [ ]:
diabetes_df_copy['result'] = pd.to_numeric(diabetes_df_copy['result'])
diabetes_df_copy['prediabetesresult'] = pd.to_numeric(diabetes_df_copy['prediabetesresult'])

In [ ]:
diabetes_df_copy.shape

In [ ]:
diabetes_df_copy.to_csv('DiabetesClean.csv', sep=',', encoding='utf-8')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

In [ ]:
diabetes_df_copy.describe()

In [ ]:

pd.crosstab( diabetes_df_copy.agegroup, diabetes_df_copy.isdiabetic)

In [ ]:
pd.crosstab( diabetes_df_copy.agegroup, diabetes_df_copy.isdiabetic, normalize = "index")

In [ ]:
sn.heatmap( diabetes_df_copy.corr(), annot=True );

In [ ]:
sn.factorplot(x="agegroup", data=diabetes_df_copy,
               size=4, kind="count", palette="muted")

In [ ]:
sn.factorplot(x="agegroup", hue = 'isdiabetic', data=diabetes_df_copy,
               size=4, kind="count", palette="muted")

In [ ]:
sn.factorplot(x="isdiabetic", data=diabetes_df_copy,
               size=4, kind="count", palette="muted")

In [ ]:
sn.factorplot(x="isdiabetic", hue = 'agegroup', data=diabetes_df_copy,
               size=4, kind="count", palette="muted")

In [ ]:
sn.barplot( x = 'age',
         y = 'isdiabetic',
         data = diabetes_df_copy,
         estimator = sum )